In [3]:
import pandas as pd
import pyro
import torch
import numpy as np
from build_features import calculate_columns

In [17]:
raw_path = "../../data/raw/"
data = pd.read_csv(raw_path + "Loan_default.csv")
data=data.drop(["LoanID"], axis=1)
calculate_columns(data)
data.head()

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,...,Default,_TotalInterest,_MonthlyPayment,_TotalPayment,_LoanToIncomeRatio,_RemainingIncomeAfterPayment,_YearsEmployed,_LoanTermYears,_InterestToPrincipalRatio,_NetIncome
0,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,...,0,27735840.36,1405.194444,27786427.36,0.588262,84588.805556,6.666667,3.0,548.28,48156.64
1,69,50432,124440,458,15,1,4.81,60,0.68,Master's,...,0,35913384.00,2074.000000,36037824.00,2.467481,48358.000000,1.250000,5.0,288.60,16138.24
2,46,84208,129188,451,26,3,21.17,24,0.31,Master's,...,1,65637839.04,5382.833333,65767027.04,1.534154,78825.166667,2.166667,2.0,508.08,58103.52
3,32,31713,44799,743,0,3,7.07,24,0.23,High School,...,0,7601494.32,1866.625000,7646293.32,1.412638,29846.375000,0.000000,2.0,169.68,24419.01
4,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,...,0,2855754.72,190.395833,2864893.72,0.447179,20246.604167,0.666667,4.0,312.48,5517.99


In [18]:
def iv_woe(data, target, bins=10, show_woe=False):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns

    valuable_features = np.array([])
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})

        
        # Calculate the number of events in each group (bin)
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        
        # Calculate % of events in each group.
        d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()

        # Calculate the non events in each group.
        d['Non-Events'] = d['N'] - d['Events']
        # Calculate % of non events in each group.
        d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()

        # Calculate WOE by taking natural log of division of % of non-events and % of events
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        d.insert(loc=0, column='Variable', value=ivars)
        print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d], axis=0)

        if d['IV'].sum()>0.02:
            valuable_features = np.append(valuable_features, ivars)

        #Show WOE Table
        if show_woe == True:
            print(d)
    return newDF, woeDF, valuable_features

In [19]:
newDF, woeDF, valuable_features = iv_woe(data,"Default")
newDF.head()
print(valuable_features)

Information value of Age is 0.28746
Information value of Income is 0.124293
Information value of LoanAmount is 0.073382
Information value of CreditScore is 0.011613
Information value of MonthsEmployed is 0.092365
Information value of NumCreditLines is 0.007902
Information value of InterestRate is 0.173103
Information value of LoanTerm is 2e-05
Information value of DTIRatio is 0.00414
Information value of Education is 0.008127
Information value of EmploymentType is 0.020633
Information value of MaritalStatus is 0.007784
Information value of HasMortgage is 0.005096
Information value of HasDependents is 0.011747
Information value of LoanPurpose is 0.005057
Information value of HasCoSigner is 0.014952
Information value of _TotalInterest is 0.147502
Information value of _MonthlyPayment is 0.043687
Information value of _TotalPayment is 0.147292
Information value of _LoanToIncomeRatio is 0.198668
Information value of _RemainingIncomeAfterPayment is 0.140086
Information value of _YearsEmployed